In [1]:
import numpy as np
import sys
import scipy

class LBM_2_Carlemann:
    def __init__(self):
        # Constants for the D1Q3 lattice (1D, 3 velocities)
        self.w = np.array([2/3, 1/6, 1/6])  # Weights for D1Q3 lattice
        self.e = np.array([0, 1, -1])  # Lattice directions: [0, +1, -1]
        self.c_s = 1 / np.sqrt(3)  # Speed of sound for D1Q3 lattice...is this the delta_x\delta_t ??
        self.g = 9.81  # Acceleration due to gravity (m/s^2)
        self.kn = 0.1 #Knudsen number, much less than 1 for chapman-enskogg expansion

        # Parameters
        self.tau = 1.0  # Relaxation time
        self.Nx = 5 # Number of grid points...my code for the F matrices makes the kernel die if this number is 81 or higher
        self.L = 0.001  # Length of the domain (in meters)
        self.delta_t = self.L/self.Nx

        # Initialize macroscopic variables: density(height) and velocity field
        self.h = np.ones(self.Nx)  # height field
        self.u = np.zeros(self.Nx)  # Velocity field

        # Initialize distribution functions (f_i), f has 3 directions (D1Q3)
        self.f = np.zeros((self.Nx, 3))  # Distribution functions for D1Q3
        self.feq = np.zeros_like(self.f)  # Equilibrium distribution functions

    def get_F_single(self):

        g=self.g
        c=self.c_s

        F1 = 1/(self.tau*self.kn) * np.array([
        [0, 1, 1 ],
        [0, (1 / (2 * c)) - 1, -(1 / (2 * c))],
        [0, -1 / (2 * c), -1*(1 - (1 / (2 * c)))]
        ])
    

        F2 = 1/(self.tau*self.kn*2*c**2)* np.array([
        [-g, -g, -g, -g , -g - 4, -g + 4, -g, -g + 4, -g - 4],
        [g, g, g, g, g + 2, g - 2, g, g - 2, g + 2],
        [g, g, g, g, g + 2, g - 2, g, g - 2, g + 2]
        ])

        F3 =np.array([
        [0, 0, 0, 0 , 2, -2, 0, -2, 2],
        [0, 0, 0, 0, -1, 1, 0, 1, -1],
        [0, 0, 0, 0, -1, 1, 0, 1, -1]
        ])

        F3 =  1/(self.tau*self.kn*2*c**2)*np.hstack([F3,F3,F3])

        return F1,F2,F3
    
    def create_one_hot_vectors(self,l):
       
        vector = np.eye(l)
        return vector


    def get_F_multiple(self,F1,F2,F3):

        F1_multiple_stacked_alpha = []
        F2_multiple_stacked_alpha = []
        F3_multiple_stacked_alpha = []
        
        delta = self.create_one_hot_vectors(self.Nx)

        for alpha in range(0,self.Nx):
        
            F1_multiple_stacked_alpha.append(np.kron(delta[alpha,:],F1)) #(33)
            F2_multiple_stacked_alpha.append(np.kron(delta[alpha,:],np.kron(delta[alpha,:],F2)))
            F3_multiple_stacked_alpha.append(np.kron(delta[alpha,:],np.kron(delta[alpha,:],np.kron(delta[alpha,:],F3))))

        # Stack matrices along the first dimension
        F1_multiple_stacked = np.vstack( F1_multiple_stacked_alpha)  #why vstack? in (33) looks like stack on columns
        F2_multiple_stacked = np.vstack( F2_multiple_stacked_alpha)  
        F3_multiple_stacked = np.vstack( F3_multiple_stacked_alpha)  

        return F1_multiple_stacked,F2_multiple_stacked,F3_multiple_stacked


    def get_A(self,F1,F2,F3):

        A11=F1
        A12=F2
        A13=F3

        I1 = np.eye(F1.shape[0])
        I2= np.eye(F2.shape[0])

        A22 = np.kron(I1,F1) + np.kron(F1,I1)
        A23 = np.kron(I2,F2) + np.kron(F2,I2)
        A33 = np.kron(I1,np.kron(I1,F1)) + np.kron(I1,np.kron(F1,I1))+ np.kron(F1,np.kron(I1,I1)) 

        return A11,A12,A13,A22,A23,A33
    
        

    def get_collision(self,A11,A12,A13,A22,A23,A33):

        zero1 = np.zeros((A22.shape[0], A11.shape[1]))  # Zero block for row 2
        zero2 = np.zeros((A33.shape[0], A11.shape[1] + A22.shape[1]))  # Zero block for row 3

        # Stack rows together
        row1 = np.hstack([A11, A12, A13])  # First row
        row2 = np.hstack([zero1, A22, A23])  # Second row
        row3 = np.hstack([zero2, A33])  # Third row
            
        # Stack all rows vertically
        C_collision = np.vstack([row1, row2, row3])

        return C_collision
    
    def get_S(self):

        main_diagonal = [1] * (self.Nx * len(self.e))
        #upper_diagonal = [-1] * ((self.Nx) * len(self.e)-3)
        lower_diagonal = [-1] * ((self.Nx) * len(self.e)-3)

        # Create the matrix
        S =  np.diag(main_diagonal, k=0)     # Main diagonal
        #S += np.diag(upper_diagonal, k=3)    # Upper diagonal 
        S += np.diag(lower_diagonal, k=-3)   # Lower diagonal 
       
        #add periodic BC
        S[0,-3]=S[1,-2]=S[2,-1]=-1
        #S[-3,0]=S[-2,1]=S[-1,2]=1
        
        #multiply with e_m
        S[::3, :] = 0 
        S[1::3, :] *= 1 
        S[2::3, :] *= -1 

        S = (1/self.delta_t)* S

        return S


    def get_B(self,S1):

        I1 = np.eye(S1.shape[0])

        B11=S1
        B22 = np.kron(I1,S1) + np.kron(S1,I1)
        B33 = np.kron(I1,np.kron(I1,S1)) + np.kron(S1,np.kron(I1,I1)) + np.kron(I1,np.kron(S1,I1))

        return B11,B22,B33
        
    
    def get_streaming(self, B11, B22, B33):

        C_streaming = scipy.linalg.block_diag(B11, B22, B33)

        return C_streaming
        
    


In [2]:
Gen = LBM_2_Carlemann()

F1_single,F2_single,F3_single  = Gen.get_F_single()
print("dim Matrix F1_single: \n", F1_single.shape)
print("dim Matrix F2_single: \n", F2_single.shape)
print("dim Matrix F3_single: \n", F3_single.shape)

F1_,F2_,F3_ = Gen.get_F_multiple(F1_single,F2_single,F3_single)
print("dim Matrix F1: \n", F1_.shape)
print("dim Matrix F2: \n", F2_.shape)
print("dim Matrix F3: \n", F3_.shape)

A11_, A12_, A13_, A22_, A23_, A33_ = Gen.get_A(F1_,F2_,F3_)
print("dim Matrix A11: \n", A11_.shape)
print("dim Matrix A12: \n", A12_.shape)
print("dim Matrix A13: \n", A13_.shape)
print("dim Matrix A22: \n", A22_.shape)
print("dim Matrix A23: \n", A23_.shape)
print("dim Matrix A33: \n", A33_.shape)

C_collision = Gen.get_collision(A11_, A12_, A13_, A22_, A23_, A33_)
print("dim Matrix Cc: \n", C_collision.shape)

###################################################################################################

S_ = Gen.get_S()
print("dim Matrix S: \n", S_.shape)

B11_,B22_,B33_= Gen.get_B(S_)
print("dim Matrix B1: \n", B11_.shape)
print("dim Matrix B2: \n", B22_.shape)
print("dim Matrix B3: \n", B33_.shape)

C_streaming = Gen.get_streaming(B11_, B22_, B33_)
print("dim Matrix Cs: \n", C_streaming.shape)

CL_LBE_Matrix = C_collision + C_streaming



dim Matrix F1_single: 
 (3, 3)
dim Matrix F2_single: 
 (3, 9)
dim Matrix F3_single: 
 (3, 27)
dim Matrix F1: 
 (15, 15)
dim Matrix F2: 
 (15, 225)
dim Matrix F3: 
 (15, 3375)
dim Matrix A11: 
 (15, 15)
dim Matrix A12: 
 (15, 225)
dim Matrix A13: 
 (15, 3375)
dim Matrix A22: 
 (225, 225)
dim Matrix A23: 
 (225, 3375)
dim Matrix A33: 
 (3375, 3375)
dim Matrix Cc: 
 (3615, 3615)
dim Matrix S: 
 (15, 15)
dim Matrix B1: 
 (15, 15)
dim Matrix B2: 
 (225, 225)
dim Matrix B3: 
 (3375, 3375)
dim Matrix Cs: 
 (3615, 3615)


In [3]:
import numpy as np
import sys

class LBM_2_Carlemann1:
    def __init__(self):
        # Constants for the D1Q3 lattice (1D, 3 velocities)
        self.w = np.array([2/3, 1/6, 1/6])  # Weights for D1Q3 lattice
        self.e = np.array([0, 1, -1])  # Lattice directions: [0, +1, -1]
        self.c_s = 1 / np.sqrt(3)  # Speed of sound for D1Q3 lattice...is this the delta_x\delta_t ??
        self.g = 9.81  # Acceleration due to gravity (m/s^2)
        self.kn = 0.1 #Knudsen number, much less than 1 for chapman-enskogg expansion

        # Parameters
        self.tau = 1.0  # Relaxation time
        self.Nx = 5  # Number of grid points...my code for the F matrices makes the kernel die if this number is 81 or higher
        self.L = 10.0  # Length of the domain (in meters)

        # Initialize macroscopic variables: density(height) and velocity field
        self.h = np.ones(self.Nx)  # height field
        self.u = np.zeros(self.Nx)  # Velocity field

        # Initialize distribution functions (f_i), f has 3 directions (D1Q3)
        self.f = np.zeros((self.Nx, 3))  # Distribution functions for D1Q3
        self.feq = np.zeros_like(self.f)  # Equilibrium distribution functions

     #return a 1D array with one non-zero element of value 1 at specified index
    def one_nonzero(self,dim, n):
        array = np.zeros((dim))
        if n>-1 and n<dim:
            array[n] = 1
        return array
    #make the F matrices for the collision matrix for n grid points
    def gen_F(self):
        f1 = np.zeros((3,3))
        f1[0,1] = f1[0,2] = 1
        f1[1,1] = 1/(2*self.c_s) - 1
        f1[1,2] = -1/(2*self.c_s)
        f1[2,1] = -1/(2*self.c_s)
        f1[2,2] = 1/(2*self.c_s) -1
        f1 = (1/(self.tau*self.kn))*f1
        
        f2 = np.zeros((3,9))
        for i in range(9):
            f2[0,i] = -self.g
            f2[1,i]  = self.g
            f2[2,i]  = self.g
        f2[0,4] = f2[0,4] -4 
        f2[0,8] = f2[0,8] -4
        f2[0,5] = f2[0,5] +4 
        f2[0,7] = f2[0,7] +4
        for i in range(2):
            f2[i+1, 4] = f2[i+1, 4] +2
            f2[i+1, 8] = f2[i+1, 8] +2

            f2[i+1, 5] = f2[i+1, 5] -2
            f2[i+1, 7] = f2[i+1, 7] -2
        f2 = (1/(2*self.tau*self.kn*self.c_s**2))*f2

        f3 = np.zeros((3,9))
        f3[0,4] = 2
        f3[0,8] = 2
        f3[0,5] = -2 
        f3[0,7] = -2
        for i in range(2):
            f3[i+1, 4] = -1
            f3[i+1, 8] = -1

            f3[i+1, 5] = 1
            f3[i+1, 7] = 1
        f3 = np.hstack((f3,f3,f3))
        f3 = (1/(2*self.tau*self.kn*self.c_s**2))*f3

        #generalise to n grid points...strategy is to stack matrices, not create matrix of matrices I think...?
        n = self.Nx
        Q = len(self.e)
        '''
        F1 = np.zeros((dim,Q, dim*Q))
        F2 = np.zeros((dim,Q, (dim**2)*(Q**2)))
        F3 = np.zeros((dim,Q, (dim**3)*(Q**3)))
        '''
        I = self.one_nonzero(n, 0)
        F1 = np.kron(I, f1)
        F2 = np.kron(np.kron(I,I) , f2)
        F3 = np.kron(np.kron(np.kron(I, I), I), f3)


        for i in range(n-1):
            I = self.one_nonzero(n, i+1)
            F1 = np.vstack((F1, np.kron(I, f1)))
            F2 = np.vstack((F2, np.kron(np.kron(I,I) , f2)))
            F3 = np.vstack((F3, np.kron(np.kron(np.kron(I, I), I), f3)))
        
        return F1,F2,F3, f1, f2, f3
        
    #make A matrices for the collision matrix
    def gen_A(self, F1,F2,F3):
        A11 = F1
        A12 = F2
        A13 = F3
        Q= len(self.e)
        n = self.Nx
        dim = n*Q
        I = np.identity(dim)
        A22 = np.kron(F1,I) + np.kron(I, F1)
        #slows down at A22, kernel dies after with Nx = 50
        A23 = np.kron(F2,I) + np.kron(I, F2)
        A33 = np.kron(np.kron(F1,I), I) + np.kron(np.kron(I,F1), I) + np.kron(np.kron(I,I), F1)

        return A11, A12, A13, A22, A23, A33
    #make collision matrix 
    def gen_collision(self, A11, A12, A13, A22, A23, A33):
        C1 = np.vstack((A11,np.zeros((A22.shape[0]+A33.shape[0],A11.shape[1]))))
        C2 = np.vstack((A12,A22,np.zeros((A33.shape[0],A12.shape[1]))))
        C3 = np.vstack((A13, A23, A33))

        Cc = np.hstack((C1,C2,C3))

        return Cc
    
    #make streaming matrix, restricted to NN, 2nd order accuracy...only makes sense if we are considering more than 1 grid point
    def gen_streaming(self):
        Q= len(self.e)
        n = self.Nx
        inv_delta = n/(2*self.L)
        dim = n*Q
        I = np.identity(dim)
        S = np.zeros((dim, dim))
        print("S shape",S.shape)
        for i in range(dim):
            #deal with edge case here...periodic or bounce back BC...here I do code for periodic
            if i<Q:
                S[i,i+Q] = inv_delta*self.e[(i%3)-1]
                S[i, (dim-Q)+i] = -inv_delta*self.e[(i%3)-1]
            elif i>dim -Q - 1:
                S[i,dim-i] = inv_delta*self.e[(i%3)-1]
                S[i,i-Q] = -inv_delta*self.e[(i%3)-1]
            else:
                S[i,i+Q] = inv_delta*self.e[(i%3)-1]
                S[i,i-Q] = -inv_delta*self.e[(i%3)-1]

        #print(S)

        B11 = S
        B22 = np.kron(S,I) + np.kron(I, S)
        B33 = np.kron(np.kron(S,I), I) + np.kron(np.kron(I,S), I) + np.kron(np.kron(I,I), S)

        C1 = np.vstack((B11,np.zeros((B22.shape[0]+B33.shape[0],B11.shape[1]))))
        C2 = np.vstack((np.zeros((B11.shape[0],B22.shape[1])),B22,np.zeros((B33.shape[0],B22.shape[1]))))
        C3 = np.vstack((np.zeros((B11.shape[0]+B22.shape[0],B33.shape[1])), B33))

        Cs = np.hstack((C1,C2,C3))

        return Cs, B11,B22,B33,S

In [4]:
Matrix_C = LBM_2_Carlemann1()
F1,F2,F3, f1,f2,f3  = Matrix_C.gen_F()

A11, A12, A13, A22, A23, A33 = Matrix_C.gen_A(F1,F2,F3)

Cc = Matrix_C.gen_collision(A11, A12, A13, A22, A23, A33)
Cs, B11,B22,B33,S = Matrix_C.gen_streaming()

S shape (15, 15)


In [5]:
are_equal = np.array_equal(f1, F1_single)
print("Are the matrices F1?", are_equal)

are_equal = np.array_equal(f2, F2_single)
print("Are the matrices F2?", are_equal)

are_equal = np.array_equal(f3, F3_single)
print("Are the matrices F3?", are_equal)


are_equal = np.array_equal(F1, F1_)
print("Are the matrices F1?", are_equal)

are_equal = np.array_equal(F2, F2_)
print("Are the matrices F2?", are_equal)

are_equal = np.array_equal(F3, F3_)
print("Are the matrices F3?", are_equal)

are_equal = np.array_equal(A11, A11_)
print("Are the matrices A11?", are_equal)

are_equal = np.array_equal(A12, A12_)
print("Are the matrices A12?", are_equal)

are_equal = np.array_equal(A13, A13_)
print("Are the matrices A13?", are_equal)

are_equal = np.array_equal(A22,A22_)
print("Are the matrices A22?", are_equal)

are_equal = np.array_equal(A23, A23_)
print("Are the matrices A23?", are_equal)

are_equal = np.array_equal(A33, A33_)
print("Are the matrices A33?", are_equal)

are_equal = np.array_equal(Cc, C_collision)
print("Are the matrices Cc?", are_equal)

are_equal = np.array_equal(S, S_)
print("Are the matrices S?", are_equal)
print(S_)

are_equal = np.array_equal(B11, B11_)
print("Are the matrices B11?", are_equal)

are_equal = np.array_equal(B22, B22_)
print("Are the matrices B22?", are_equal)

are_equal = np.array_equal(B33,B33_)
print("Are the matrices B33?", are_equal)


are_equal = np.array_equal(Cs, C_streaming)
print("Are the matrices Cs?", are_equal)




Are the matrices F1? True
Are the matrices F2? True
Are the matrices F3? True
Are the matrices F1? True
Are the matrices F2? True
Are the matrices F3? True
Are the matrices A11? True
Are the matrices A12? True
Are the matrices A13? True
Are the matrices A22? True
Are the matrices A23? True
Are the matrices A33? True
Are the matrices Cc? True
Are the matrices S? False
[[    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.]
 [    0.  5000.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0. -5000.     0.]
 [    0.     0. -5000.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.  5000.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.]
 [    0. -5000.     0.     0.  5000.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.]
 [    0.     0.  5000.     0.     0. -5000.     0.     0.     0.     0.
      0.

In [6]:
def embed_matrix(C, delta_t, num_steps):
    """
    Embeds a matrix C into a larger matrix A with specified properties.

    Parameters:
        C (np.ndarray): The matrix to embed (must be square).
        delta_t (float): The length of each time step.
        num_steps (int): The number of time steps (and thus of cascaded blocks in A).

    Returns:
        np.ndarray: The constructed matrix A.
    """
    # Validate inputs
    if not (isinstance(C, np.ndarray) and C.ndim == 2 and C.shape[0] == C.shape[1]):
        raise ValueError("C must be a square matrix.")

    # Identity matrix with the same size as C
    Id = np.eye(C.shape[0])

    # Compute -O = -(Id + delta_t * C)
    O = -(Id + delta_t * C)

    # Size of the large matrix A
    A_size = num_steps * C.shape[0]

    # Initialize A as a zero matrix
    A = np.zeros((A_size, A_size))

    # Fill in the diagonal blocks
    for i in range(num_steps):
        # Main diagonal (Identity blocks)
        start_idx = i * C.shape[0]
        A[start_idx:start_idx + C.shape[0], start_idx:start_idx + C.shape[0]] = Id

        # Secondary diagonal (-O blocks)
        if i > 0:
            prev_idx = (i - 1) * C.shape[0]
            A[start_idx:start_idx + C.shape[0], prev_idx:prev_idx + C.shape[0]] = O

    return A

In [7]:
N_t = 3  # Number of time steps
delta_t = Gen.delta_t
Lin_Euler_Matrix = embed_matrix(CL_LBE_Matrix, delta_t, N_t)

In [12]:
initial_distribution = np.kron(Gen.h,(1/6, 2/3, 1/6))  # Function of 3 values in the 1D case
print(initial_distribution.shape)

(15,)


In [ ]:
f1 = initial_distribution
f2 = np.kron(f1,f1)
f3 = np.kron(f2,f1)

def create_concatenated_vector(N, f1, f2, f3):
    # Repeat f1 3*N times
    repeated_f1 = np.tile(f1, N)
    # Repeat f2 9*N^2 times
    repeated_f2 = np.tile(f2, (N**2))
    # Repeat f3 27*N^3 times
    repeated_f3 = np.tile(f3, (N**3))
    # Concatenate all repeated arrays
    concatenated_vector = np.concatenate((repeated_f1, repeated_f2, repeated_f3))
    return concatenated_vector

phi_t0 = np.hstack((f1,f2,f3))

#phi_t0 = create_concatenated_vector(Gen.Nx, f1, f2, f3)

print(phi_t0.shape)

def append_zeros(f, N):
    # Ensure `f` is a NumPy array
    f = np.array(f)

    # Create N zero vectors of the same shape as `f`
    zero_vector = np.zeros_like(f)
    zeros_to_append = np.tile(zero_vector, (N-1,))

    # Concatenate `f` with the appended zeros
    result = np.concatenate([f, zeros_to_append])

    return result

phi = append_zeros(phi_t0, N_t)
print(phi.shape[0])

(3615,)
10845


In [18]:
Inverted_matrix = np.linalg.inv(Lin_Euler_Matrix)
x = np.dot(Inverted_matrix, phi)

In [ ]:
def extract_phi(x, subvector_dim, N):
    if len(x) < N * subvector_dim:
        raise ValueError("The length of x is too small for the given N and subvector_dim.") 
    # Extract phi components
    phi = [x[i * subvector_dim : (i + 1) * subvector_dim] for i in range(N)]
    return phi

def phi_truncation(phi_list, N):
    num_values = 3 * N
    truncated_phi = [phi[:num_values] for phi in phi_list]
    return truncated_phi

def divide_truncated_phi(truncated_phi, N):
    result = []
    for phi in truncated_phi:
        # Ensure the truncated phi has at least 3 * N elements
        if len(phi) < 3 * N:
            raise ValueError("Each truncated phi must have at least 3 * N elements.")
        
        # Divide phi into N groups of 3
        groups = [phi[3 * i : 3 * (i + 1)] for i in range(N)]
        result.extend(groups)
    
    return result 


In [20]:
# x = np.arange(10845)  # Example: [0, 1, 2, ..., 10844]

N_time = 3
N_grid = Gen.Nx
subvector_dim = 3 * N_grid + 9 * N_grid**2 + 27 * N_grid**3
print("Subvector dimension:", subvector_dim)
# Extract phi components for each time step
phi = extract_phi(x, subvector_dim, N_time)
# Extract the first 3 * N values from each phi_i
truncated_phi = phi_truncation(phi, N_grid)
# Divide each truncated phi into N groups of 3 and combine them
three_dim_vectors = divide_truncated_phi(truncated_phi, N_grid)
# Display the results
for i, vec in enumerate(three_dim_vectors, start=1):
    print(f"Vector {i}: {vec}")

Subvector dimension: 3615
Vector 1: [0.16666667 0.66666667 0.16666667]
Vector 2: [0.16666667 0.66666667 0.16666667]
Vector 3: [0.16666667 0.66666667 0.16666667]
Vector 4: [0.16666667 0.66666667 0.16666667]
Vector 5: [0.16666667 0.66666667 0.16666667]
Vector 6: [0.15336833 0.68103936 0.18030731]
Vector 7: [0.14190333 0.69412936 0.19339731]
Vector 8: [0.10847333 0.72555936 0.22482731]
Vector 9: [0.13865333 0.69575436 0.19502231]
Vector 10: [0.15361833 0.68091436 0.18018231]
Vector 11: [0.14077103 0.69486368 0.1931485 ]
Vector 12: [0.11771024 0.73432542 0.20668024]
Vector 13: [0.04305245 0.82322894 0.25890376]
Vector 14: [0.10773812 0.69799816 0.25614299]
Vector 15: [0.13892699 0.68199336 0.21020819]
